# Set environment

In [1]:
source("config_path.R")
source("util_path.R")

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,


# Import Data

In [2]:
dat_go_cne_h99 <- readr::read_tsv(file.path(INFODIR, "go_all_cne_h99_Fungidb.tsv"))
#dat_go_cne_h99 <- dat_go_cne_h99 %>% select(-X10)
dat_go_cne_h99 <- dat_go_cne_h99 %>% 
    rename(
        gene = `[Gene ID]`,
        source_id = `[source_id]`,
        name = `[Gene Name or Symbol]`,
        go_bp_curated_id   = `[Curated GO Processes]`,
        go_bp_curated_name = `[Curated GO Process IDs]`,
        go_cc_curated_id   = `[Curated GO Component IDs]`,
        go_cc_curated_name = `[Curated GO Components]`,
        go_fc_curated_id   = `[Curated GO Function IDs]`,
        go_fc_curated_name = `[Curated GO Functions]`,
        #ec_num = `[EC numbers]`,
        org = `[Organism]`,
        go_bp_comp_id   = `[Computed GO Process IDs]`,
        go_bp_comp_name = `[Computed GO Processes]`,
        go_cc_comp_id   = `[Computed GO Component IDs]`,
        go_cc_comp_name = `[Computed GO Components]`,
        go_fc_comp_id   = `[Computed GO Function IDs]`,
        go_fc_comp_name = `[Computed GO Functions]`)

head(dat_go_cne_h99)

Warning message:
“Missing column names filled in: 'X19' [19]”Parsed with column specification:
cols(
  `[Gene ID]` = col_character(),
  `[source_id]` = col_character(),
  `[Gene Name or Symbol]` = col_character(),
  `[Organism]` = col_character(),
  `[Computed GO Component IDs]` = col_character(),
  `[Computed GO Components]` = col_character(),
  `[Computed GO Function IDs]` = col_character(),
  `[Computed GO Functions]` = col_character(),
  `[Computed GO Process IDs]` = col_character(),
  `[Computed GO Processes]` = col_character(),
  `[Curated GO Component IDs]` = col_character(),
  `[Curated GO Components]` = col_character(),
  `[Curated GO Function IDs]` = col_character(),
  `[Curated GO Functions]` = col_character(),
  `[Curated GO Process IDs]` = col_character(),
  `[Curated GO Processes]` = col_character(),
  `[EC numbers]` = col_character(),
  `[EC numbers from OrthoMCL]` = col_character(),
  X19 = col_character()
)


gene,source_id,name,org,go_cc_comp_id,go_cc_comp_name,go_fc_comp_id,go_fc_comp_name,go_bp_comp_id,go_bp_comp_name,go_cc_curated_id,go_cc_curated_name,go_fc_curated_id,go_fc_curated_name,go_bp_curated_name,go_bp_curated_id,[EC numbers],[EC numbers from OrthoMCL],X19
CNAG_00001,CNAG_00001-t26_1,N/A,C. neoformans var. grubii H99,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NA
CNAG_00002,CNAG_00002-t26_1,N/A,C. neoformans var. grubii H99,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,3.5.1.- (In linear amides.),NA
CNAG_00003,CNAG_00003-t26_1,N/A,C. neoformans var. grubii H99,GO:0016021,integral component of membrane,N/A,N/A,GO:0055085,transmembrane transport,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NA
CNAG_00004,CNAG_00004-t26_1,N/A,C. neoformans var. grubii H99,GO:0005634,nucleus,"GO:0003677, GO:0008270","DNA binding, zinc ion binding",GO:0006351,"transcription, DNA-templated",N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NA
CNAG_00005,CNAG_00005-t26_1,N/A,C. neoformans var. grubii H99,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,3.1.1.- (Carboxylic ester hydrolases.),NA
CNAG_00006,CNAG_00006-t26_1,N/A,C. neoformans var. grubii H99,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NA


In [3]:
preprocess <- function(dat, varname){
    varname <- rlang::sym(varname)
    dat     <- dat %>% 
        select(gene, !!varname) %>% 
        filter(!!varname != 'N/A') %>% 
        select(gene, !!varname) %>%
        separate_rows(!!varname, sep = ",") %>% 
        distinct
    return(dat)
}
#varname <- "go_bp_comp_id"
#varname <- rlang::sym(varname)
#dat_go_bp <- dat_go_cne_h99 %>% select(gene, !!varname) %>% filter(!!varname != 'N/A') %>% distinct
#head(dat_go_bp)
dat_go_bp <- preprocess(dat_go_cne_h99, "go_bp_comp_id")
dat_go_fc <- preprocess(dat_go_cne_h99, "go_fc_comp_id")
dat_go_cc <- preprocess(dat_go_cne_h99, "go_cc_comp_id")
head(dat_go_bp)
head(dat_go_fc)
head(dat_go_cc)

gene,go_bp_comp_id
CNAG_00003,GO:0055085
CNAG_00004,GO:0006351
CNAG_00007,GO:0006431
CNAG_00007,GO:0006418
CNAG_00010,GO:0030001
CNAG_00010,GO:0055085


gene,go_fc_comp_id
CNAG_00004,GO:0003677
CNAG_00004,GO:0008270
CNAG_00007,GO:0005524
CNAG_00007,GO:0004812
CNAG_00007,GO:0004825
CNAG_00007,GO:0000166


gene,go_cc_comp_id
CNAG_00003,GO:0016021
CNAG_00004,GO:0005634
CNAG_00007,GO:0005737
CNAG_00010,GO:0016020
CNAG_00018,GO:0005634
CNAG_00022,GO:0005739


In [4]:
write.table(data.frame(dat_go_bp$go_bp_comp_id), 
            file = file.path(INFODIR, "go_id_bp.txt"), 
            row.names = FALSE, col.names = FALSE, 
            quote = FALSE)

write.table(data.frame(dat_go_fc$go_fc_comp_id), 
            file = file.path(INFODIR, "go_id_fc.txt"), 
            row.names = FALSE, col.names = FALSE, 
            quote = FALSE)

write.table(data.frame(dat_go_cc$go_cc_comp_id), 
            file = file.path(INFODIR, "go_id_cc.txt"), 
            row.names = FALSE, col.names = FALSE, 
            quote = FALSE)

# go info

In [5]:
dat_go_info_bp <- readr::read_tsv(file.path(INFODIR, "go_info_bp_YeastMine.tsv"), col_names = FALSE)
colnames(dat_go_info_bp) <- c("id", "name", "type", "desc")

dat_go_info_fc <- readr::read_tsv(file.path(INFODIR, "go_info_fc_YeastMine.tsv"), col_names = FALSE)
colnames(dat_go_info_fc) <- c("id", "name", "type", "desc")

dat_go_info_cc <- readr::read_tsv(file.path(INFODIR, "go_info_cc_YeastMine.tsv"), col_names = FALSE)
colnames(dat_go_info_cc) <- c("id", "name", "type", "desc")

head(dat_go_info_bp, 3)
head(dat_go_info_fc, 3)
head(dat_go_info_cc, 3)

Parsed with column specification:
cols(
  X1 = col_character(),
  X2 = col_character(),
  X3 = col_character(),
  X4 = col_character()
)
Parsed with column specification:
cols(
  X1 = col_character(),
  X2 = col_character(),
  X3 = col_character(),
  X4 = col_character()
)
Parsed with column specification:
cols(
  X1 = col_character(),
  X2 = col_character(),
  X3 = col_character(),
  X4 = col_character()
)


id,name,type,desc
GO:0000002,mitochondrial genome maintenance,biological_process,The maintenance of the structure and integrity of the mitochondrial genome; includes replication and segregation of the mitochondrial chromosome.
GO:0000045,autophagosome assembly,biological_process,"The formation of a double membrane-bounded structure, the autophagosome, that occurs when a specialized membrane sac, called the isolation membrane, starts to enclose a portion of the cytoplasm."
GO:0000077,DNA damage checkpoint,biological_process,"A cell cycle checkpoint that regulates progression through the cell cycle in response to DNA damage. A DNA damage checkpoint may blocks cell cycle progression (in G1, G2 or metaphase) or slow the rate at which S phase proceeds."


id,name,type,desc
GO:0000030,mannosyltransferase activity,molecular_function,"Catalysis of the transfer of a mannosyl group to an acceptor molecule, typically another carbohydrate or a lipid."
GO:0000049,tRNA binding,molecular_function,Interacting selectively and non-covalently with transfer RNA.
GO:0000062,fatty-acyl-CoA binding,molecular_function,"Interacting selectively and non-covalently with acyl-CoA, any derivative of coenzyme A in which the sulfhydryl group is in thiolester linkage with a fatty acyl group."


id,name,type,desc
GO:0000015,phosphopyruvate hydratase complex,cellular_component,"A multimeric enzyme complex, usually a dimer or an octamer, that catalyzes the conversion of 2-phospho-D-glycerate to phosphoenolpyruvate and water."
GO:0000124,SAGA complex,cellular_component,"A SAGA-type histone acetyltransferase complex that contains Spt8 (in budding yeast) or a homolog thereof; additional polypeptides include Spt group, consisting of Spt7, Spt3, and Spt20/Ada5, which interact with the TATA-binding protein (TBP); the Ada group, consisting of Ada1, Ada2, Ada3, Ada4/Gcn5, and Ada5/Spt20, which is functionally linked to the nucleosomal HAT activity; Tra1, an ATM/PI-3 kinase-related protein that targets DNA-bound activators for recruitment to promoters; the TBP-associated factor (TAF) proteins, consisting of Taf5, Taf6, Taf9, Taf10, and Taf12, which mediate nucleosomal HAT activity and are thought to help recruit the basal transcription machinery; the ubiquitin specifc protease Ubp-8."
GO:0000139,Golgi membrane,cellular_component,The lipid bilayer surrounding any of the compartments of the Golgi apparatus.


In [6]:
dat <- dat_go_bp
dat_info <- dat_go_info_bp
dat <- dat %>% 
    rename(id = go_bp_comp_id) %>%
    filter(id != 'N/A') %>% 
    select(gene, id) %>%
    separate_rows(id, sep = ",") %>% 
    left_join(dat_info, by = "id") %>% 
    mutate(label = paste(id, name, sep = " | ")) %>%
    na.omit
dat_go_bp_annot <- dat

dat <- dat_go_fc
dat_info <- dat_go_info_fc
dat <- dat %>%
    rename(id = go_fc_comp_id) %>%
    filter(id != 'N/A') %>% 
    select(gene, id) %>%
    separate_rows(id, sep = ",") %>% 
    left_join(dat_info, by = "id") %>% 
    mutate(label = paste(id, name, sep = " | ")) %>%
    na.omit
dat_go_fc_annot <- dat

dat <- dat_go_cc
dat_info <- dat_go_info_cc
dat <- dat %>%
    rename(id = go_cc_comp_id) %>%
    filter(id != 'N/A') %>% 
    select(gene, id) %>%
    separate_rows(id, sep = ",") %>% 
    left_join(dat_info, by = "id") %>% 
    mutate(label = paste(id, name, sep = " | ")) %>%
    na.omit
dat_go_cc_annot <- dat

head(dat_go_bp_annot, 3)
head(dat_go_fc_annot, 3)
head(dat_go_cc_annot, 3)

gene,id,name,type,desc,label
CNAG_00003,GO:0055085,transmembrane transport,biological_process,"The process in which a solute is transported across a lipid bilayer, from one side of a membrane to the other.",GO:0055085 | transmembrane transport
CNAG_00004,GO:0006351,"transcription, DNA-templated",biological_process,The cellular synthesis of RNA on a template of DNA.,"GO:0006351 | transcription, DNA-templated"
CNAG_00007,GO:0006431,methionyl-tRNA aminoacylation,biological_process,"The process of coupling methionine to methionyl-tRNA, catalyzed by methionyl-tRNA synthetase. The methionyl-tRNA synthetase is a class-I synthetase. The activated amino acid is transferred to the 2'-OH group of a methionine-accetping tRNA. The 2'-O-aminoacyl-tRNA will ultimately migrate to the 3' position via transesterification.",GO:0006431 | methionyl-tRNA aminoacylation


gene,id,name,type,desc,label
CNAG_00004,GO:0003677,DNA binding,molecular_function,Any molecular function by which a gene product interacts selectively and non-covalently with DNA (deoxyribonucleic acid).,GO:0003677 | DNA binding
CNAG_00007,GO:0005524,ATP binding,molecular_function,"Interacting selectively and non-covalently with ATP, adenosine 5'-triphosphate, a universally important coenzyme and enzyme regulator.",GO:0005524 | ATP binding
CNAG_00009,GO:0005509,calcium ion binding,molecular_function,Interacting selectively and non-covalently with calcium ions (Ca2+).,GO:0005509 | calcium ion binding


gene,id,name,type,desc,label
CNAG_00003,GO:0016021,integral component of membrane,cellular_component,The component of a membrane consisting of the gene products and protein complexes having at least some part of their peptide sequence embedded in the hydrophobic region of the membrane.,GO:0016021 | integral component of membrane
CNAG_00004,GO:0005634,nucleus,cellular_component,"A membrane-bounded organelle of eukaryotic cells in which chromosomes are housed and replicated. In most cells, the nucleus contains all of the cell's chromosomes except the organellar chromosomes, and is the site of RNA synthesis and processing. In some species, or in specialized cell types, RNA metabolism or DNA replication may be absent.",GO:0005634 | nucleus
CNAG_00007,GO:0005737,cytoplasm,cellular_component,"All of the contents of a cell excluding the plasma membrane and nucleus, but including other subcellular structures.",GO:0005737 | cytoplasm


In [7]:
dat <- dat_go_bp_annot
gset_go_bp <- split(dat$gene, dat$label)

dat <- dat_go_fc_annot
gset_go_fc <- split(dat$gene, dat$label)

dat <- dat_go_cc_annot
gset_go_cc <- split(dat$gene, dat$label)


head(gset_go_bp, 2)
head(gset_go_fc, 2)
head(gset_go_cc, 2)

$`GO:0000002 | mitochondrial genome maintenance`
[1] "CNAG_04267"

$`GO:0000045 | autophagosome assembly`
[1] "CNAG_07645"

$`GO:0000030 | mannosyltransferase activity`
[1] "CNAG_00996" "CNAG_03832" "CNAG_04763" "CNAG_06834"

$`GO:0000049 | tRNA binding`
[1] "CNAG_02122" "CNAG_03005" "CNAG_05827"

$`GO:0000015 | phosphopyruvate hydratase complex`
[1] "CNAG_03072" "CNAG_06868"

$`GO:0000124 | SAGA complex`
[1] "CNAG_01396" "CNAG_04512"

In [8]:
outfile <- file.path(OUTDIR, "HTS-GO-Info.RData")

save(
    dat_go_cne_h99,
    
    dat_go_info_bp,
    dat_go_info_fc,
    dat_go_info_cc,
    
    dat_go_bp_annot,
    dat_go_fc_annot, 
    dat_go_cc_annot, 
    
    gset_go_bp,
    gset_go_fc,
    gset_go_cc,
    
    file = outfile)

tools::md5sum(outfile)

/shared_space/TA_clint/analysis_output/out/HTS-GO-Info.RData 
                          "ab425c3f0909746bea5b0d229dff9ff7"